## CV19 - Dlib 문제점과 YOLO
### Dlib 얼굴 인식의 문제점
얼굴 인식 방법
1. Dlib 라이브러리 이용하여 face landmark 찾기  
   얼굴 인식 저하, 동영상 처리시 프레임 저하, 얼굴 각도, 방향, 크기 등 변화에 취약 등등 문제 발생  
   얼굴 인식에 있어서 작은 디바이스에서 처리할 경우 작은 모델이 필요하다.  
   통상 작은 모델은 성능이 떨어진다.
   https://tech.kakaoenterprise.com/63
   https://github.com/clovaai/EXTD_Pytorch
   
sliding window 방식을 버려야 한다.  
요구조건
병렬화가 가능해야 한다.  
apple은 coreml 라이브러리 지원  
https://developer.apple.com/documentation/coreml  
http://machinethink.net/blog/ios-11-machine-learning-for-everyone/  
gpu가 지원한다. 아이폰 뉴럴엔진  
하드웨어가 지원하지 않으면 소프트웨어로 가능?  
https://www.slideshare.net/inureyes/ml-kit-machine-learning-sdk  
https://www.tensorflow.org/lite?hl=ko  
tflite도 안되면? 직접 프로그래밍  
SIMD: https://stonzeteam.github.io/SIMD-%EB%B3%91%EB%A0%AC-%ED%94%84%EB%A1%9C%EA%B7%B8%EB%9E%98%EB%B0%8D/  
OpenCL: https://www.khronos.org/opencl/  
OpenGL ES: https://developer.android.com/guide/topics/graphics/opengl?hl=ko  
### Single Stage Object Detection
object detection: https://lilianweng.github.io/lil-log/2018/12/27/object-detection-part-4.html  
single shot object detectors: https://jonathan-hui.medium.com/what-do-we-learn-from-single-shot-object-detectors-ssd-yolo-fpn-focal-loss-3888677c5f4d  
번역: https://murra.tistory.com/17  
### YOLO v1의 등장
https://youtu.be/NM6lrxy0bxs?t=721  
### YOLO v1의 원리
RCNN: region proposal network, 물체가 있을만한 곳에 박스를 그린다.  
YOLO v1: grid 중심, grid 각 셀에 물체가 있을 것이다.  
한 개의 셀에 2개의 bbox와 20개의 클래스를 예측하고자 할 때 최종 출력 개수는?  
7x7 feature map일 경우 7x7x(5xB+C)가 된다. 즉 (bbox좌표와 confidence)x box개수 + class개수  
YOLO의 inference 과정  
7x7 grid 마지막 layer 해석  
classification은 P(real|pred)인 likelihood 사용  
https://www.slideshare.net/TaegyunJeon1/pr12-you-only-look-once-yolo-unified-realtime-object-detection  
confidence score: P(class|object) * P(object) * IoU  
7x7x2개의 class confidence score가 계산  
### YOLO v1의 성능
YOLO v1의 loss함수  
논문에서는 mAP 63.4, CVPR2016 발표에서는 69.0으로 상승  
Faster RCNN과 성능이 비슷하면서 속도는 6배 이상  
YOLO v1의 단점  
각 셀에서 하나의 클래스만 예측하므로 작은 object에 대한 예측이 어려움  
bbox 분산이 너무 넓어 새로운 형태의 bbox 예측이 잘 안됨  
backbone을 거친 feature map으로 bbox 예측하기 때문에 localization이 다소 부정확함  
### YOLO v2
1년 뒤 CVPR2017에서 v2 발표  
Make it better  
Do it faster
Makes us stronger  
9000개의 class를 예측하면서 detection  
https://dhhwang89.tistory.com/136  
### YOLO v3
원리 참조 사이트: https://taeu.github.io/paper/deeplearning-paper-yolov3/  
### SSD 의 특징
SSD: Single Shot MultiBox Detector  
YOLO v1이후 1stage detector에 대한 방법이 발전, YOLO v1에서 gird를 사용해서 생기는 단점을 해결할 수 있는 방법 제시  
Image Pyramid  
![Image_Pyramid](https://aiffelstaticprd.blob.core.windows.net/media/images/GC-10-L-30.max-800x600.png)
* imagenet으로 pretrained VGG16사용
* VGG에서 pooling을 거친 block은 하나의 이미지로 사용 가능, YOLO v1은 7x7만 사용, 38x38,19x19,10x10,.. multi scale
* 각 feature map에 따라 grid 크기가 다르므로 small object도 detection 가능

다만 feature map 개수만큼 연산량이 많아지며 초기 feature map의 경우 box를 계산하기엔 network가 충분히 깊지 않다.  
Workflow  
box를 예측하기 위한 기본적인 정보가 없기 때문에 넓게 bbox를 파악할 수 없다. 성능 손실 가능성 존재  
anchor를 이용할 필요가 있다.  object가 발견된 bbox를 재조정하는 layer를 추가한다, default box == anchor  
![nn](https://aiffelstaticprd.blob.core.windows.net/media/images/GC-10-L-31.max-800x600.png)
(a) : 이미지, GT 데이터셋  
(b) : (vgg backbone 에 가까운) fine-grained feature map. 8x8 grid 에서 각각의 grid 에 3개 anchor box 를 적용할 수 있음. 고양이는 크기가 작기 때문에 (a) 의 고양이는 8x8 feature map 내 grid 중 1개의 anchor box 로 부터 학습될 수 있음  
(c) : 개의 경우 크고 세로로 긴 경향을 보이기 때문에 receptive field 가 넓은 4x4 feature map 이 사용됨  
![nn](https://aiffelstaticprd.blob.core.windows.net/media/original_images/GC-10-L-32.png)![nn](https://aiffelstaticprd.blob.core.windows.net/media/images/GC-10-L-33.max-800x600.png)![nn](https://aiffelstaticprd.blob.core.windows.net/media/images/GC-10-L-34.max-800x600.png)  
https://docs.google.com/presentation/d/1rtfeV_VmdGdZD5ObVVpPDPIODSDxKnFSU0bsN_rgZXc/pub?start=false&loop=false&delayms=3000&slide=id.g179f601b72_0_51  
### SSD의 loss와 성능
SSD Loss function  
https://seongkyun.github.io/papers/2019/07/01/SSD/  
SSD 성능
![nn](https://aiffelstaticprd.blob.core.windows.net/media/images/GC-10-L-45.max-800x600.png)  
FCOS  
https://hoya012.github.io/blog/ICCV-2019-paper-preview/  
https://blog.naver.com/jinyuri303/221876480557  
### Face Detect
SSD을 활용하여 face detection  
S3FD: https://seongkyun.github.io/papers/2019/03/21/S3FD/
      https://arxiv.org/abs/1708.05237  
DSFD: https://arxiv.org/pdf/1810.10220.pdf  
RetinaFace: https://arxiv.org/pdf/1905.00641.pdf  